In [8]:
import numpy as np
import pandas as pd


def bit_length(arr):
    return np.frexp(arr)[1]


empty = [-1, -1]

In [9]:
SUBS = {
    "0": "₀",
    "1": "₁",
    "2": "₂",
    "3": "₃",
    "4": "₄",
    "5": "₅",
    "6": "₆",
    "7": "₇",
    "8": "₈",
    "9": "₉",
}


def subscript(num):
    return "".join(SUBS[d] for d in str(num))


def varname(i, var):
    return f"{var}{subscript(i)}"


def x_to_bits(x, width):
    bits = [int(b) for b in bin(x)[2:]]
    return [0] * (width - len(bits)) + bits

In [10]:
S = np.array(
    [0b0000, 0b0001, 0b0010, 0b0100, 0b1000, 0b0011, 0b0111, 0b0101, 0b1001],
    dtype=np.uint8
)

matrix = np.array(
    [
        # S1     S2      S3      S4      S5      S6      S7      S8      S9
        [[3, 2], empty,  empty,  [4, 2], [6, 3], empty,  empty,  empty,  empty ], # S1
        [[2, 3], empty,  empty,  empty,  empty,  [7, 1], empty,  empty,  empty ], # S2
        [[5, 1], empty,  empty,  empty,  empty,  empty,  empty,  empty,  empty ], # S3
        [empty,  empty,  empty,  empty,  empty,  empty,  empty,  [5, 3], empty ], # S4
        [empty,  empty,  empty,  empty,  empty,  empty,  empty,  empty,  [2, 3]], # S5
        [empty,  empty,  [0, 0], empty,  empty,  empty,  [6, 2], empty,  empty ], # S6
        [empty,  empty,  empty,  empty,  empty,  empty,  empty,  [1, 3], empty ], # S7
        [empty,  [2, 3], empty,  empty,  empty,  empty,  empty,  empty,  empty ], # S8
        [empty,  [5, 0], empty,  empty,  empty,  empty,  empty,  empty,  empty ], # S9
    ],
    dtype=np.int32,
)

In [ ]:
valid = np.all(~(matrix == empty), axis=2)

pairs = matrix[valid]

state_bits = bit_length(np.max(S))
input_bits = bit_length(pairs[:, 0])

transitions = []

for i, row in enumerate(matrix):
    for j, col in enumerate(row):
        if (col == empty).all():
            continue

        s_bits = x_to_bits(S[i], state_bits)
        s_next_bits = x_to_bits(S[j], state_bits)
        x_bits = x_to_bits(col[0], input_bits)

        transitions.append((s_bits, s_next_bits, x_bits))

df = pd.DataFrame(
    [
        {
            "S(t)": "".join(map(str, s_bits)),
            "S(t+1)": "".join(map(str, s_next_bits)),
            "x": "".join(map(str, x_bits)),
        }
        for s_bits, s_next_bits, x_bits in transitions
    ]
)

In [12]:
df

,S(t),S(t+1),x
0,0000,0000,011
1,0000,0100,100
2,0000,1000,110
3,0001,0000,010
4,0001,0011,111
5,0010,0000,101
6,0100,0101,101
7,1000,1001,010
8,0011,0010,000
9,0011,0111,110


In [13]:
def build_sr_functions(transitions):
    bit_width = state_bits + input_bits

    S = [[] for _ in range(state_bits)]
    R = [[] for _ in range(state_bits)]

    for s_bits, s_next_bits, x_bits in transitions:
        for i in range(state_bits):
            term = ""

            for j in range(bit_width):
                inv = False

                if j < state_bits:
                    var = varname(j + 1, "z")
                    inv = s_bits[j] == 0
                else:
                    var = varname(j + 1 - state_bits, "x")
                    inv = x_bits[j - state_bits] == 0

                if inv:
                    term += "~"

                term += var

            if s_bits[i] == 0 and s_next_bits[i] == 1:
                S[i].append(term)
            elif s_bits[i] == 1 and s_next_bits[i] == 0:
                R[i].append(term)
            else:
                continue

    return zip(S, R)

In [14]:
for i, (S, R) in enumerate(build_sr_functions(transitions)):
    print(
        varname(i + 1, "S")
        + " = "
        + " | ".join(S)
        + "\n"
        + varname(i + 1, "R")
        + " = "
        + " | ".join(R)
        + "\n"
    )

S₁ = ~z₁~z₂~z₃~z₄x₁x₂~x₃
R₁ = z₁~z₂~z₃z₄x₁~x₂x₃

S₂ = ~z₁~z₂~z₃~z₄x₁~x₂~x₃ | ~z₁~z₂z₃z₄x₁x₂~x₃
R₂ = ~z₁z₂~z₃z₄~x₁x₂~x₃

S₃ = ~z₁~z₂~z₃z₄x₁x₂x₃
R₃ = ~z₁~z₂z₃~z₄x₁~x₂x₃ | ~z₁z₂z₃z₄~x₁~x₂x₃

S₄ = ~z₁z₂~z₃~z₄x₁~x₂x₃ | z₁~z₂~z₃~z₄~x₁x₂~x₃
R₄ = ~z₁~z₂~z₃z₄~x₁x₂~x₃ | ~z₁~z₂z₃z₄~x₁~x₂~x₃

